# Learn To Synchronize Videos

## Model declaration

In [4]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.optim import lr_scheduler
from sync_net import reset_first_and_last_layers, TripletNet, TripletLoss
from data_loader import get_dataset
from torch.utils.data import DataLoader
from trainer import fit
cuda = torch.cuda.is_available()

torch.cuda.set_device(0)
embedding_net = models.resnet50(pretrained=True)
reset_first_and_last_layers(embedding_net)
model = TripletNet(embedding_net)
model.cuda(0)
model = nn.DataParallel(model).cuda()
lr = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_fn = TripletLoss(margin=0.5)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
n_epochs = 20
log_interval = 100
start_epoch = 0
training_path = r"C:\Users\root\Projects\VideoSynchronizationWithPytorch\trainings\base"

## Load dataset

In [ ]:
training_path = r'\\primnis.gi.polymtl.ca\dfs\cheriet\Images\Cardiologie\Angiographie'
validation_path = r'\\primnis.gi.polymtl.ca\dfs\cheriet\Images\Cardiologie\Angiographie\KR-11'
dataset = get_dataset(training_path, validation_path)

## Load training state

In [ ]:
load_state_path = training_path + r"\training_state_0.pth"
state = torch.load(load_state_path)

start_epoch = int(state['epoch']) + 1
model.load_state_dict(state['model'])
optimizer.load_state_dict(state['optimizer'])
scheduler.load_state_dict(state['scheduler'])

## Train

In [6]:
torch.cuda.empty_cache()
train_loader = DataLoader(dataset, batch_size=20, shuffle=True, num_workers=4)
fit(train_loader, None, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval)

Starting Epoch 0
Will sample from train_loader
Train: [0/74480 (0%)]	Loss: 4.990984	Elapsed time: 0:00:05.524216
Train: [2000/74480 (3%)]	Loss: 0.377423	Elapsed time: 0:00:52.592256
Train: [4000/74480 (5%)]	Loss: 0.101146	Elapsed time: 0:01:40.480636
Train: [6000/74480 (8%)]	Loss: 0.075056	Elapsed time: 0:02:27.386536
Train: [8000/74480 (11%)]	Loss: 0.066325	Elapsed time: 0:03:15.211294
Train: [10000/74480 (13%)]	Loss: 0.060601	Elapsed time: 0:04:02.719750
Train: [12000/74480 (16%)]	Loss: 0.051565	Elapsed time: 0:04:51.361483
Train: [14000/74480 (19%)]	Loss: 0.044950	Elapsed time: 0:05:41.954396
Train: [16000/74480 (21%)]	Loss: 0.044364	Elapsed time: 0:06:32.214145
Train: [18000/74480 (24%)]	Loss: 0.044186	Elapsed time: 0:07:22.715011
Train: [20000/74480 (27%)]	Loss: 0.045051	Elapsed time: 0:08:13.135588
Train: [22000/74480 (30%)]	Loss: 0.024351	Elapsed time: 0:09:03.476781
Train: [24000/74480 (32%)]	Loss: 0.030342	Elapsed time: 0:09:53.208583
Train: [26000/74480 (35%)]	Loss: 0.042547	

KeyboardInterrupt: 

## Test trained model